### Brenton Grundman
#### 829460164

# Facial and Species Division

## Preliminary Notes to Self

For nottingham faces: for each face, alter image to 442x539 by adding white rows equivalently at the top and bottom with emphasis on the bottom, and left and right with emphasis on the left, by dividing the difference in rows and columns from the maximum by 2.  This will help create more centralized images between which to make comparisons.

Need to print out the trained "ghost image" of the faces overlapping.  Also try to figure out the traits that previous layer may correspond to in case of the neural network.  

## Used Links

http://pics.stir.ac.uk/2D_face_sets.htm

https://archive.ics.uci.edu/ml/datasets/Grammatical+Facial+Expressions#

http://www.robots.ox.ac.uk/~vgg/data/pets/

http://scikit-learn.org/stable/auto_examples/decomposition/plot_faces_decomposition.html

## Necessary Code

In order to train in the three alternate methods, we will need NN, LDA, and QDA code.  Further, we will need certain imports, as well as additional tangential code that has been used in the past.  Instead of putting all of the tangential code here, it is included in the other python code.

mlutils is giving me some problems and after various attempts at fixing it, my best guess is that it's a problem with using the python 2 distribution instead of the 3 that I typically use.  The error that it's throwing is causing "Run All" to fail repeatedly which is getting really annoying - I may remove it entirely and just hope this does not negatively affect the project.

### Imports

In [3]:
import numpy as np
#import mlutils as ml
import neuralnetworks as nn

SyntaxError: invalid syntax (mlutils.py, line 102)

In [4]:
import qdalda as ql

### Original Code

In [5]:
def trainLDA(X,T,parameters = 0):
    lda = ql.LDA()
    lda.train(X,T)
    return lda
def evaluateLDA(model,X,T):
    predicted,prob,d = model.use(X)
    percentCorrect = 100*(np.sum(T==predicted)/float(T.shape[0]))
    return percentCorrect
def trainNN(X,T,parameters):
    nnet = nn.NeuralNetworkClassifier(X.shape[1],parameters[0],len(np.unique(T)))
    return nnet.train(X, T, nIterations=parameters[1])
def evaluateNN(model,X,T):
    predict = model.use(X)
    percentCorrect = 100*(np.sum(T == predict)/float(T.shape[0]))
    return percentCorrect

In [6]:
def trainQDA(X,T,parameters = 0):
    qda = ql.QDA()
    qda.train(X,T)
    return qda
def evaluateQDA(model,X,T):
    predicted,prob,d = model.use(X)
    percentCorrect = 100*(np.sum(T==predicted)/float(T.shape[0]))
    return percentCorrect

## Useful for coding

resizing images:

resize_contain(image, size, validate=True)
Resize the image so that it can fit in the specified area, keeping the ratio and without crop. It’s the same behavior as css background-size: contain property. A white a background border is created.

Resize the image to minimum so that it is contained in a 200x100 rectangle is the ratio between source and destination image.

from PIL import Image
from resizeimage import resizeimage

fd_img = open('test-image.jpeg', 'r')
img = Image.open(fd_img)
img = resizeimage.resize_contain(img, [200, 100])
img.save('test-image-contain.jpeg', img.format)
fd_img.close()

printing:

from IPython.display import Image
Image(img/picture.png)

for retrieving image names from file:

import pandas
io = pandas.read_csv('test.csv',sep=",",usecols=(1,2,4)) # To read 1st,2nd and 4th columns
print io 

## Method

First, I take in and resize all the images.  The images will be related to either male or female (or cat or dog) using the pixels directly.  There is a corresponding .txt file that has two values, the name of the file and a 0 for female or a 1 for male.  These are the classification values.  Via the training, an image should be created which is a ghostly representation of the training faces for each male and female, which will be generated and printed and subsequently used for testing the NN, LDA, and QDA against the faces.

Alteration: Iterate through the files; any file that starts with an f is a female image, m is male.  Use this for training instead of creating a file.  The next code accomplishes this.  This makes the code more adaptable as all that is needed to add face images would be an appropriate naming style.

As I write this code and attempt different approaches, I will take notes of the problems I'm running into (as I'm worried that my lack of knowledge of Python will have, now that I've had to restart, not left me enough time to figure out how to fix the problems I'm running into).  This will help me keep track of what I've attempted in the past and show my thought process as to how to fix the problems I'm running into.  Hopefully I'll get to a point of being able to test and evaluate this code but we'll see.

In [7]:
import os
import PIL.Image
#from resizeimage import resizeimage
from IPython.display import Image
from scipy.misc import imread
import scipy.misc
 
path = './faces'
listing = os.listdir(path)
male_image = np.zeros((539,442,3))
lin_mal = male_image.flatten()

female_image = np.zeros((539,442,3))
lin_fem = female_image.flatten()

males = 0
females = 0

#Commented code used to generate various filetypes as original filetype (gif) does not work in Jupyter
#for infile in listing:
#    base_file, ext = os.path.splitext(infile)
#    #if ext == ".jpg":
#    #    os.rename('./faces/'+infile, './faces/' + base_file + '.gif')
#    im = PIL.Image.open('./faces/' + infile).convert('RGB').resize((442,539))
#    im.save('./faces/' + base_file + '.jpg')
for infile in listing:
    #fd_img = PIL.Image.open('./faces/' + infile,'r').resize((442,539))
    base_file, ext = os.path.splitext(infile)
    #img = PIL.Image.open(fd_img)
    #img = resizeimage.resize_contain(img,[442,539])
    #img.save(infile,img.format)
#This code was intended to print all images in the code output block but was unsuccessful
#    if ext == '.png':
#        Image('./faces/' + infile)
    #with open(('./faces/'+infile),'rb') as f:
    #    display(Image(f.read()), format='png')
    #![title](faces/f084.gif)
    print "current file is: " + infile
    #This true/false will be used to train the algorithms - it won't have the filename, so this should be okay
    print infile.startswith('m')
    #Convert image to string of values and average it with others of this type
    #img_str = ' '.join(map(str,fd_img.flatten().tolist()))
    #This gets an image into an array
    img = imread('./faces/' + infile)
    
    #This flattens the image into a 1 dimensional array then turns it into a string if I need that
    lin_img = img.flatten()
    pixel_list = lin_img.tolist()
    pixel_str_list = map(str, pixel_list)
    img_str = ' '.join(pixel_str_list)

    #X is now a string of pixel values
    X = lin_img
    #print lin_img
    #print lin_mal
    #print X
    T = infile.startswith('m')
#    model = trainLDA(X,T)
    if T == True and ext == ".jpg":
        #for val in lin_mal:
        #    lin_mal(val) = lin_mal(val) + lin_img(val)
        #print len(lin_mal)
        #print len(lin_img)
        lin_mal = lin_mal + lin_img
        print lin_mal
        males = males + 1
    elif T == False and ext == ".jpg":
        lin_fem = lin_fem + lin_img
        print lin_fem
        females = females + 1
#        for val in img:
#            female_image(val) = female_image(val) + img(val)
#            females = females + 1
#for val in male_image:
#    male_image(val) = male_image(val)/males
#for val in female_image:
#    female_image(val) = female_image(val)/females
lin_mal = lin_mal/males
lin_fem = lin_fem/females
print lin_mal
print lin_fem
male_image = np.reshape(lin_mal, (539,442,3))
female_image = np.reshape(lin_fem, (539,442,3))
#male_image.save("./faces/MALE.jpg",male_image.format)
scipy.misc.imsave('./faces/MALE.jpg',male_image)
scipy.misc.imsave('./faces/FEMALE.jpg',female_image)
#print male_image
#print female_image
maleTar = np.matrix(lin_mal)
X = maleTar
T = 1
model = trainLDA(X,T)

current file is: f005.jpg
False
[ 239.  239.  239. ...,   59.   59.   59.]
current file is: f084.gif
False
current file is: f033.jpg
False
[ 494.  494.  494. ...,  314.  314.  314.]
current file is: f085.gif
False
current file is: f038.jpg
False
[ 749.  749.  749. ...,  569.  569.  569.]
current file is: m074.jpg
True
[ 132.  132.  132. ...,  255.  255.  255.]
current file is: f059.png
False
current file is: f019.gif
False
current file is: f084.png
False
current file is: m093.gif
True
current file is: m030.png
True
current file is: m042.png
True
current file is: m093.jpg
True
[ 387.  387.  387. ...,  505.  505.  505.]
current file is: f005.png
False
current file is: m096.png
True
current file is: m083.gif
True
current file is: f087.gif
False
current file is: f011.gif
False
current file is: m023.png
True
current file is: m070.jpg
True
[ 642.  642.  642. ...,  531.  531.  531.]
current file is: m036.jpg
True
[ 897.  897.  897. ...,  784.  784.  784.]
current file is: m078.png
True
curren

qdalda.py:24: RuntimeWarning: invalid value encountered in divide
  Xs = (X - self.means) / self.stds


ValueError: shapes (1,1) and (714714,1) not aligned: 1 (dim 1) != 714714 (dim 0)

Example of an image from this set:

![image](./faces/f084.jpg)

### Captain's Log

Getting stuck in this second attempt at this - running out of time to repeat the experiment, coding, and preparation.

Attempts this time through have been to create a ghost image by hand for each the male and female image sets and use that for training - however, averaging the images is creating difficulties.  Further, sending the pixels directly into the LDA, QDA, and NN is also for some reason causing trouble this time.  I believe it is due to some alteration that I made a while back but can't seem to remember.

For some reason, I'm getting 'can't assign to function call' while trying to create an average of ints to create the ghost images.  I've tried various fixes for this (flattening the image to a 1D array, converting to string).

Tried another fix.  No luck... ValueError: operands could not be broadcast together with shapes (238238,) (714714,) 
Apparently (if I'm right), this means that I can't directly add two arrays together because they're different sizes.  I would expect this with the .gif images, but this error occurred with a .jpg image which I specifically went through and saved in the resized form.  This means they probably did not resize correctly, which means that directly working with the pixels will not work until I figure this out.  It's late and I'm burned out... hopefully tomorrow I can get this working.

I've had what I think is a revelation - I've been using Python 3 at home, which I believe is part of why some of what I've been attempting hasn't worked.  I hope to find workarounds soon.

WOW. Okay. Yeah, forgot that since I had to convert it to RGB, I'd have 3 values for each grayscale pixel, so my image would be 3 times the size I originally thought.  Now I just have to convert my 1D array to a 2D array and into a .jpg image and I'll have an average of the male faces (since I was working on that first).

![image](./faces/MALE.jpg)

Getting an image now.  Not a ghost image... just a bunch of horizontal lines in a rgb pattern.  Still figuring it out.

Well, it's an elongated grayscale image now.  Not quite there.  Not getting a face in any respect here.

And now it's compact grayscale madness.  I think I have my axes flipped.

I'm getting something a little better I think.  It's about the same size as the woman above and has some density differences in the image, but it's RGB.  Still trying different things.

WHOA! Success!  The above image is the ghost male image.  Next is female.

![image](./faces/FEMALE.jpg)

What a banshee. Much more discernible face in the male image.  This is probably a testament to males having a more defined appearance and women being more distinct - or simply due to differences in photography.  

I keep getting an error "too many indices for array".  I hope this doesn't prevent me from running these pixels or else this will have been a waste of time.

Now I keep getting various shape errors.  Pretty stuck again.  I keep wanting to train the male ghost face against the value 1 and female against 0, or train a group of male faces against the male ghost face (and same for female), or train a group of male faces against 1 and females against 0 but none of these approaches are cooperating with me.

## Results

Unfortunately, I haven't been able to get the shape errors worked out in time.  Theoretically I'd be able to train these face images against the ghost images and against a 1 or a 0 in NN, LDA, and QDA.  The latter two would give means of binary classification between male and female faces, and to what degree the classification was successful.  NN would have been used more to derive what "features" (in this case, pixel location intensity density) appeared where most commonly in male and female faces.  My predictions going off the ghost images above is that the male faces would likely have the eyes as an especially learned feature.  Surprisingly the women would not likely have had any features stand out - I would have expected hair to make a difference between the two, but the background of the male images gets rid of this possibility.

I expect we would have seen something more discernible come out of the female faces if these images had been in color.  I'm fairly disappointed in my lack of ability to get the tests working and I believe that the experiment is close enough that it could be tweaked slightly and then expanded within an hour by someone who wanted to see the expected results come to life.  Unfortunately, time is not permitting.

## Conclusion

The experiment was a failure.  It was more successful the first time through with the approach to the experiment fresh in my mind, and the tools I'd been used to, with more time (and particularly free time, with finals abound) to work out the kinks.  Unfortunately I had never completed the analysis of the previous tests, and when I lost the data and had to restart, it was already too late.  So while this experiment was a failure, I fully believe it is near enough to completion that it could be repeated or expanded and be successful.

It is worth noting that for some reason, even with mlutils commented out, each cell must be run individually because mlutils will prevent the rest of the code cells from running to completion.  For anyone attempting to expand or re-run the project, please make note of this and either run the cells individually or fix the problem mlutils before running the experiment.

Thank you for reading my final entry.

~Brent